# Imports

In [315]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, jaccard_similarity_score
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline


# Split dataset to train and test

In [316]:
categories = ['sci.space', 'comp.graphics', 'talk.politics.misc', 'rec.sport.hockey', 'comp.sys.mac.hardware'] 
remove = ('headers', 'footers', 'quotes')
# twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories = categories, remove = remove )
# twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42, categories = categories, remove = remove )
train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories = categories, remove = remove )
test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42, categories = categories, remove = remove )



# Train Profiles

## Create single train documents object with docs and targets

In [317]:
documents = tuple(zip(train.data, train.target))

In [318]:
# for doc in documents:
#     print(doc[1])

## Vectorise train data to: split docs by words, remove stop words.

In [319]:
vect = CountVectorizer(stop_words = 'english')
vect.fit(train.data)
# train_data = vect.transform(train.data)
# test_data = vect.transform(test.data)
# vectores.append((train_data, test_data))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

## Create general words list

In [320]:
words_list = vect.get_feature_names()
print(words_list[5000])


attacked


## For each word in train set calculete number of documents

In [321]:
%%time

words_list_count = {}
for doc in documents: 
    for word in words_list:
        if word in doc[0]:
            if word in words_list_count:
                if doc[1] in words_list_count[word]:
                    words_list_count[word][doc[1]] += 1
                else:
                    words_list_count[word][doc[1]] = 1
            else:
                words_list_count[word] = {}
                words_list_count[word][doc[1]] = 1
                
            

CPU times: user 1min 17s, sys: 324 ms, total: 1min 17s
Wall time: 1min 18s


In [322]:
# example of calulation result
words_list_count['mac']

{0: 38, 1: 108, 2: 11, 3: 21, 4: 14}

## Calculate total number of docs for each class

In [323]:
total_docs_count = {}
for doc in documents:
    if doc[1] in total_docs_count:
        total_docs_count[doc[1]] += 1
    else:
        total_docs_count[doc[1]] = 1

In [324]:
total_docs_count.items()

dict_items([(3, 593), (2, 600), (4, 465), (1, 578), (0, 584)])

## Build Jaccard Profiles 

In [325]:
%%time

def build_profiles(profile_length, classes, words_list_count, total_docs_count):
    classes = classes
    profiles = {}
    for p_class in classes: 
        k_profile = {}
        for word in words_list_count: 
            if p_class in words_list_count[word]:
                a = words_list_count[word][p_class]
                word_classes = words_list_count[word]
                del word_classes[p_class]
                b = 0
                for item in word_classes:
                    b += word_classes[item]
                c = total_docs_count[p_class] - a
                word_jac = a/(a+b+c)
                if word not in k_profile:
                    k_profile[word] = word_jac
                else:
                    print('This word already exists in the profile')
        k_profile = sorted(k_profile.items(), key=lambda item: item[1], reverse=True)
        k_profile = k_profile[:profile_length]
        profiles[p_class] = dict(k_profile)
    return profiles
    

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10 µs


In [326]:
%%time
profiles = build_profiles(profile_length=100, classes=[0, 1, 2, 3, 4], words_list_count=words_list_count, total_docs_count=total_docs_count)

CPU times: user 125 ms, sys: 4.75 ms, total: 130 ms
Wall time: 132 ms


In [327]:
profiles.keys()

dict_keys([0, 1, 2, 3, 4])

In [336]:
profiles[4]

{'ab': 0.5763440860215053,
 'ac': 0.6236559139784946,
 'ad': 0.5978494623655914,
 'ai': 0.6494623655913978,
 'al': 0.8172043010752689,
 'ar': 0.853763440860215,
 'ate': 0.5741935483870968,
 'ati': 0.578494623655914,
 'av': 0.621505376344086,
 'ave': 0.5655913978494623,
 'ca': 0.6860215053763441,
 'ce': 0.6946236559139785,
 'ch': 0.6344086021505376,
 'ct': 0.6086021505376344,
 'ea': 0.789247311827957,
 'ec': 0.6086021505376344,
 'ed': 0.8365591397849462,
 'ee': 0.6817204301075269,
 'el': 0.7032258064516129,
 'em': 0.6236559139784946,
 'en': 0.8688172043010752,
 'ent': 0.6645161290322581,
 'er': 0.9139784946236559,
 'ere': 0.6150537634408603,
 'es': 0.8623655913978494,
 'et': 0.7096774193548387,
 'ev': 0.5741935483870968,
 'fo': 0.6946236559139785,
 'ge': 0.6279569892473118,
 'ha': 0.8731182795698925,
 'hat': 0.7720430107526882,
 'hi': 0.8172043010752689,
 'ho': 0.7333333333333333,
 'ia': 0.5483870967741935,
 'ic': 0.7569892473118279,
 'id': 0.6645161290322581,
 'ig': 0.5591397849462365,

# Test Profiles

## Vectorize each document

In [329]:
test_docs_targets = dict(zip(test.data, test.target))


In [330]:
test.data[0]

"\nIt's not quite what you were asking, but a few years ago I helped some EE\nremote sensing people run some experiments on the microwave emmissivity of\nice; they used the sky for a background calibration source.  They said that\nfrom Earth's surface the sky looks like a 60K blackbody.\n"

In [331]:
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf.fit(test.data)
doc_tfidf = tfidf.transform(test.data)
tfidf_list = dict(zip(tfidf.get_feature_names(), np.ravel(doc_tfidf.sum(axis=0))))



    
    

## Predict class for each doc

In [332]:
%%time

test_prdiction = []
for doc in test.data:
    w_doc = [0,0]
    for profile, jaccard_data in profiles.items(): 
        p_w = 0
        for word, tfidff_score in tfidf_list.items():
            if word in doc:
                if word in jaccard_data:
                    p_w += tfidff_score * jaccard_data[word]
        if p_w > w_doc[1]:
            w_doc[1] = p_w
            w_doc[0] = profile
    test_prdiction.append(w_doc)

CPU times: user 3min 29s, sys: 450 ms, total: 3min 30s
Wall time: 3min 30s


In [333]:
test_prdiction

[[4, 42.002530770956696],
 [4, 65.749771843825911],
 [4, 25.032166612923724],
 [4, 67.25355434501148],
 [4, 44.02419818713512],
 [4, 58.075024391474308],
 [4, 49.847562329447015],
 [4, 66.652333059716383],
 [4, 47.602364340565487],
 [4, 17.930370646852957],
 [4, 55.830669455022196],
 [4, 32.684041571347777],
 [4, 11.785727556159426],
 [4, 46.69899068451415],
 [4, 0.39163383505680455],
 [4, 35.620216783351594],
 [4, 49.599221443858077],
 [4, 63.548097653286128],
 [4, 64.112211942131651],
 [4, 23.415458343707218],
 [4, 55.194750578086236],
 [4, 57.446277515448074],
 [4, 64.629084874707701],
 [4, 64.022351065548079],
 [4, 59.500624510370919],
 [4, 61.172946164789195],
 [4, 27.610080757927211],
 [4, 39.596614114802328],
 [4, 59.946096510152685],
 [4, 34.619800441918528],
 [4, 28.487771412221168],
 [4, 63.84279665143746],
 [4, 45.422531338087893],
 [4, 68.06053753490265],
 [4, 67.68267237367759],
 [4, 63.968688629846902],
 [4, 17.083961488729081],
 [4, 60.360538930808282],
 [4, 53.745576935